In [21]:
# make sure this sits in same directory as
# a folder called 'csv' with 'names.csv', 'objects.csv', and 'rules.csv',
# a folder called 'new_csv' with 'new_names.csv', 'new_objects.csv', and 'new_rules.csv',
# a 'test.txt' file

import pandas as pd
import numpy as np
import copy
import os

unfound_dns = "NXDOMAIN"
no_answer = "No answer"

def pd_rows_len(csv_addr):
    dataframe = pd.read_csv(csv_addr)
    rows = dataframe.index
    num_rows = len(rows)
    return (dataframe, rows, num_rows)

In [22]:
# used later to print out dictionaries and arrays in a nice format

def print_dict(dictionary, name):
    print(f'{name}:')
    for key in dictionary.keys():
        print("{}{}".format(key.ljust(27), dictionary[key]))
    print("")
    
def print_arr(arr, name):
    print(f'{name}:')
    for elem in arr:
        print(elem)
    print("")

In [23]:
# helper functions for identifying host names & IPs
def is_hostname(word):
    dot_endings = [".edu", ".net", ".com", ".org", ".gov"]
    return (word[-4:] in dot_endings)

def is_ip(word):
    nums = word.split(".")
    if (len(nums) != 4):
        return False
    for num in nums:
        try:
            x = int(num)
        except:
            return False
    return True

In [24]:
# various global dictionaries used throughout
dns_records = {}
dns_pings = {}

In [17]:
# doing nslookup and ping on hostnames & ips

# gets string word which is IP address or hostname 
# and returns hostname or IP that resulted from `nslookup {word}`
def nslookup(word):
    input_is_hostname = is_hostname(word)
    filename = 'test.txt'
    os.system('nslookup ' + word + ' > ' + filename) 
    
    file = open(filename)
    lines = file.readlines()
    num_lines = len(lines)
    
    # looked up hostname
    if (input_is_hostname):
        # lookup returned 'No answer'
        if (num_lines < 7):
            return no_answer
        # lookup was successful
        for i in range(5, 7):
            words = lines[i].split(": ")
            if words[0] == ("Address"):
                return words[1].strip("\n")
        return no_answer
    
    # looked up ip
    # ip lookup that returned 'No answer'
    if (num_lines > 15):
        return no_answer
   
    # ip lookup successful
    if (num_lines > 5):
        words = lines[4].split()
        return words[-1][:-1]
    
    # ip lookup not sucessful
    return unfound_dns

# pings
def pings(word):
    ping_file = 'test.txt'
    os.system('ping -c 1 ' + word + ' > ' + ping_file)
    file = open(ping_file)
    lines = file.readlines()
    num_packets_line = 3
    if (is_hostname(word)):
        num_packets_line = 4
    if (len(lines) <= 1 or (len(lines) >=3 and lines[3][23:32] == "0 packets")):        
        return False
    return True

In [12]:
def get_nslookup(word):
    if word in dns_records:
        return dns_records[word]
    else:
        lookup_res = nslookup(word)
        dns_records[word] = lookup_res
        dns_records[lookup_res] = word
        return lookup_res
        
def get_ping(word):
    if word in dns_pings:
        return dns_pings[word]
    elif (word == no_answer or word == unfound_dns):
        return True
    else:
        ping_res = pings(word)
        dns_pings[word] = ping_res
        return ping_res

def comment_on_dns(word1=None, word2=None):
    nslookup_comment = ""
    ping_comment = ""
    ip = ""
    hostname = ""
    ip_given = False
    hostname_given = False
    #first input is ip
    if not word1 == None and is_ip(word1):
        ip = word1
        ip_given = True
    #first input is hostname
    if not word1 == None and is_hostname(word1):
        hostname = word1
        hostname_given = True
    #second input is ip
    if not word2 == None and is_ip(word2):
        ip = word2
        ip_given = True
    #second input is hostname
    if not word2 == None and is_hostname(word2):
        hostname = word2
        hostname_given = True
        
    # ip was given
    if ip_given:
        # either getting past nslookup result from dictionary or nslookup now
        lookup_ip_res = get_nslookup(ip)
        # handling nslookup cases
        if (lookup_ip_res == unfound_dns):
            nslookup_comment += "server can't find " + ip + "\n"
        elif (lookup_ip_res == no_answer):
            nslookup_comment += "'No answer' returned for nslookup " + ip + "\n"
        elif (hostname_given and lookup_ip_res != hostname):
            nslookup_comment += ip + " matched with " + lookup_ip_res + "\n"
            
        #either getting past ping result from dictionary or ping now
        if (get_ping(ip) == False):
            ping_comment += ip + " doesn't ping\n"
        if (get_ping(lookup_ip_res) == False):
            ping_comment += lookup_ip_res + " doesn't ping\n"
        
    
    #hostname was given
    if hostname_given:
        # either getting past nslookup result from dictionary or nslookup now
        lookup_name_res = get_nslookup(hostname)
        # handling nslookup cases
        if (lookup_name_res == no_answer):
            nslookup_comment += "'No answer' returned for nslookup " + name + "\n"
        elif (ip_given and lookup_name_res != ip):
            nslookup_comment += name + " matched with " + lookup_name_res + "\n"
        
        #either getting past ping result from dictionary or ping now
        if not (ip_given and ip == lookup_name_res):
            if (get_ping(hostname) == False):
                ping_comment += hostname + " doesn't ping\n"
            if (get_ping(lookup_name_res) == False):
                ping_comment += lookup_name_res + " doesn't ping\n"
            
    return (nslookup_comment, ping_comment)

In [7]:
# names csv
(names_pd, names_rows, names_num_rows) = pd_rows_len('./csv/names.csv')

name_nslookup_comments = [""]*names_num_rows
name_ping_comments = [""]*names_num_rows

# takes line from names.csv file and returns array of (ip, name)
def parse_name(name):
    ip_name = name[5:].split(" ")
    return (ip_name[0], ip_name[1])

# converting info from names.csv into dictionaries
# and nslook on each IP & host name 
for i in names_rows:
    line = names_pd.loc[i, "Names"]
    print(line)
    (ip, name) = parse_name(line)
    
    (nslookup_c, ping_c) = comment_on_dns(ip, name)
    name_nslookup_comments[i] = nslookup_c
    name_ping_comments[i] = ping_c

names_pd.insert(1, 'nslookup comments', name_nslookup_comments)
names_pd.insert(2, 'ping comments', name_ping_comments)
    
print_dict(dns_records, "DNS")
print_dict(dns_pings, "pings")

# print_dict(ip_pings, "IP pings")
# print_dict(name_pings, "name pings")
print_arr(name_nslookup_comments, "nslookup comments")
print_arr(name_ping_comments, "ping comments")
names_pd.to_csv('./new_csv/new_names.csv')

name 136.167.12.117 wo11.bc.edu
name 136.167.15.27 testhillside.bc.edu
name 136.167.12.124 aki.bc.edu
name 136.167.45.46 access.bc.edu
name 136.167.15.46 knuckleboom.bc.edu
name 136.167.44.54 athvolw71.bc.edu
name 136.167.44.67 athvolw72.bc.edu
name 136.167.44.228 atholyw71.bc.edu
name 136.167.44.141 atholyw75.bc.edu
name 136.167.44.95 athmbbw76.bc.edu
name 136.167.44.63 wbbdragon.bc.edu
name 136.167.49.9 getz.bc.edu
name 136.167.1.250 vendorvpn.bc.edu
name 136.167.173.254 isr.bc.edu
name 136.167.49.252 physics.bc.edu
name 136.167.12.104 nesn-edge4.bc.edu
name 136.167.117.249 new-cs-legacy.bc.edu
name 136.167.1.12 duffman.bc.edu
name 136.167.172.47 magnetometers-new.bc.edu
name 136.167.241.235 bcpd.bc.edu
name 136.167.45.251 caritasvpn2.bc.edu
name 136.167.63.249 caritasvpn3.bc.edu
name 136.167.54.82 roz.bc.edu
name 136.167.12.110 nesn-ibm.bc.edu
DNS:
136.167.12.117             wo11.bc.edu
wo11.bc.edu                136.167.12.117
136.167.15.27              eagleaccess.bc.edu
eagleacce

In [14]:
import copy

# converting info from objects

(objects_pd, objects_rows, objects_num_rows) = pd_rows_len('./csv/objects.csv')

objects_nslookup_comments = [""]*objects_num_rows
objects_ping_comments = [""]*objects_num_rows

# takes line from objects.csv file and 
# returns first word (object, host, range, etc) and ip/hostname
def parse_obj_line(object_line):
    words = object_line.split()
    constructor = words[0]
    constructor_arg = ""
    if constructor == "object":
        lastword = words[-1]
        if (lastword[0:3] == "obj"):
            constructor_arg = lastword.split("-")[1]
        else:
            constructor_arg = words[2]
    elif constructor == "subnet" or constructor == "host" or constructor == "range":
        constructor_arg = words[1]
    elif constructor == "network-object":
        if is_ip(words[1]):
            constructor_arg = words[1]
        else:
            constructor_arg = words[2]
    else:
        constructor_arg = None
    return (constructor, constructor_arg)

(buffer, curr_obj_arg) = parse_obj_line(objects_pd.loc[0, "Objects"])
curr_obj_index = 0
curr_host_or_subnet = ""

for i in objects_rows:
    line = objects_pd.loc[i, "Objects"]
    print("\n" + line)
    (constructor, constructor_arg) = parse_obj_line(line)
#     print(f'{constructor} {constructor_arg}')
    
    # handling object cases
    if constructor == "object":
        curr_obj_arg = constructor_arg
        curr_obj_index = i
    
    # handling subnet or host or range cases
    elif constructor == "subnet" or constructor == "host" or constructor =="range":
        curr_attr_arg = constructor_arg
        
        # if parent object and child subnet/host/range has same constructors
        if (curr_obj_arg == curr_attr_arg):
#             print(f'parent and child are same: {curr_obj_arg}')
            (nslookup_comment, ping_comment) = comment_on_dns(curr_obj_arg)
            objects_nslookup_comments[curr_obj_index] += nslookup_comment
            objects_ping_comments[curr_obj_index] += ping_comment
        
        # if the parent object and child subnet/host has differing constructors
        # check that they're corresponding IP & name pair
        if (curr_obj_arg != curr_attr_arg):
#             print(f'parent: {curr_obj_arg}, child: {curr_attr_arg}')
            (nslookup_comment, ping_comment) = comment_on_dns(curr_obj_arg, curr_attr_arg)
            objects_nslookup_comments[curr_obj_index] += nslookup_comment
            objects_ping_comments[curr_obj_index] += ping_comment
    
    # handling network-object
    elif constructor == "network-object":
        (nslookup_comment, ping_comment) = comment_on_dns(constructor_arg)
        objects_nslookup_comments[i] += nslookup_comment
        objects_ping_comments[i] += ping_comment
    else:
        continue
    print(objects_nslookup_comments[i])
    print(objects_ping_comments[i])

print_dict(dns_records, "DNS")
print_dict(dns_pings, "pings")

print_arr(objects_nslookup_comments, "nslookup comments")
print_arr(objects_ping_comments, "ping comments")
            
objects_pd.insert(1, 'nslookup comments', objects_nslookup_comments)
objects_pd.insert(2, 'ping comments', objects_ping_comments)

objects_pd.to_csv('./new_csv/new_objects.csv')


object network obj-10.31.0.0



subnet 10.31.0.0 255.255.224.0



object network obj-136.167.9.179



host 136.167.9.179



object network obj-136.167.9.128



subnet 136.167.9.128 255.255.255.240



object network obj-10.6.32.0



subnet 10.6.32.0 255.255.248.0



object network obj-10.6.40.0



subnet 10.6.40.0 255.255.248.0



object network obj-10.17.0.0



subnet 10.17.0.0 255.255.0.0



object network obj-10.18.0.0



subnet 10.18.0.0 255.255.0.0



object network obj-10.20.0.0



subnet 10.20.0.0 255.255.0.0



object network isr.bc.edu



host 136.167.173.254



description Created during name migration

object network wo11.bc.edu



host 136.167.12.117



description Created during name migration

object network testhillside.bc.edu



host 136.167.15.27



description Created during name migration

object network athvolw71.bc.edu



host 136.167.44.54



description Created during name migration

object network athvolw72.bc.edu



host 136.167.44.67



description Created dur

server can't find 136.167.42.0

136.167.42.0 doesn't ping


network-object 136.167.46.0 255.255.254.0
server can't find 136.167.46.0

136.167.46.0 doesn't ping


network-object 136.167.50.0 255.255.254.0
server can't find 136.167.50.0

136.167.50.0 doesn't ping


network-object 136.167.52.0 255.255.254.0
server can't find 136.167.52.0

136.167.52.0 doesn't ping


network-object 136.167.64.0 255.255.254.0
server can't find 136.167.64.0

136.167.64.0 doesn't ping


network-object 136.167.74.0 255.255.254.0
server can't find 136.167.74.0

136.167.74.0 doesn't ping


network-object 136.167.100.0 255.255.254.0
server can't find 136.167.100.0

136.167.100.0 doesn't ping


network-object 136.167.108.0 255.255.254.0
server can't find 136.167.108.0

136.167.108.0 doesn't ping


network-object 136.167.112.0 255.255.254.0
server can't find 136.167.112.0

136.167.112.0 doesn't ping


network-object 136.167.232.0 255.255.254.0
server can't find 136.167.232.0

136.167.232.0 doesn't ping


network-ob


54.201.95.41 doesn't ping
ec2-54-201-95-41.us-west-2.compute.amazonaws.com doesn't ping


network-object host 54.201.188.13

54.201.188.13 doesn't ping
ec2-54-201-188-13.us-west-2.compute.amazonaws.com doesn't ping


network-object host 54.201.177.49

54.201.177.49 doesn't ping
ec2-54-201-177-49.us-west-2.compute.amazonaws.com doesn't ping


network-object host 54.87.168.188

54.87.168.188 doesn't ping
ec2-54-87-168-188.compute-1.amazonaws.com doesn't ping


network-object host 52.87.67.17

52.87.67.17 doesn't ping
ec2-52-87-67-17.compute-1.amazonaws.com doesn't ping


network-object host 54.89.117.187

54.89.117.187 doesn't ping
ec2-54-89-117-187.compute-1.amazonaws.com doesn't ping


object-group network law-westlaw-printers

description printers used for westlaw in the law school

network-object host 136.167.161.104
server can't find 136.167.161.104

136.167.161.104 doesn't ping


network-object host 136.167.161.108

136.167.161.108 doesn't ping
mxlawlab155r3.bc.edu doesn't ping




server can't find 4.79.231.208

4.79.231.208 doesn't ping


network-object 4.79.171.224 255.255.255.240
server can't find 4.79.171.224

4.79.171.224 doesn't ping


network-object 24.104.159.112 255.255.255.240

24.104.159.112 doesn't ping
24-104-159-112-static.hfc.comcastbusiness.net doesn't ping


network-object 50.206.97.48 255.255.255.240
server can't find 50.206.97.48

50.206.97.48 doesn't ping


network-object 4.53.84.112 255.255.255.240
server can't find 4.53.84.112

4.53.84.112 doesn't ping


network-object 24.104.159.96 255.255.255.240

24.104.159.96 doesn't ping
24-104-159-96-static.hfc.comcastbusiness.net doesn't ping


network-object 209.170.91.16 255.255.255.240
server can't find 209.170.91.16

209.170.91.16 doesn't ping


network-object 4.59.53.16 255.255.255.240
server can't find 4.59.53.16

4.59.53.16 doesn't ping


network-object 24.104.159.128 255.255.255.240

24.104.159.128 doesn't ping
24-104-159-128-static.hfc.comcastbusiness.net doesn't ping


network-object 4.71.1

136.167.36.228 doesn't ping








10.6.0.0 doesn't ping

10.17.0.0 doesn't ping

10.18.0.0 doesn't ping

10.20.0.0 doesn't ping

10.31.0.0 doesn't ping

136.167.0.0 doesn't ping

136.167.8.0 doesn't ping

136.167.9.224 doesn't ping

136.167.9.248 doesn't ping

136.167.9.192 doesn't ping

136.167.12.0 doesn't ping

136.167.32.0 doesn't ping

136.167.37.0 doesn't ping

136.167.38.0 doesn't ping

136.167.40.0 doesn't ping

136.167.48.0 doesn't ping

136.167.64.0 doesn't ping

136.167.128.0 doesn't ping

136.167.11.0 doesn't ping

136.167.17.0 doesn't ping

136.167.18.0 doesn't ping

136.167.20.0 doesn't ping

136.167.24.0 doesn't ping

136.167.28.0 doesn't ping


70.88.192.145 doesn't ping
mailbox.aegisenergyservices.com doesn't ping


40.68.93.81 doesn't ping

216.19.230.50 doesn't ping
hge7-50.hge.net doesn't ping

13.93.117.34 doesn't ping


136.167.225.251 doesn't ping
2000-cogen.bc.edu doesn't ping

136.167.235.253 doesn't ping
hlc-m118b-cogen-1.bc.edu doesn't ping


















In [18]:
# converting info from rules
(rules_pd, rules_rows, rules_num_rows) = pd_rows_len('./csv/rules.csv')

# name_comments = ["host names:\n"]*rules_num_rows
# ip_comments = ["IPs:\n"]*rules_num_rows
# rules_comments = [""]*rules_num_rows
rules_nslookup_comments = [""]*rules_num_rows
rules_ping_comments = [""]*rules_num_rows

# returns pair of 2 lists:
# all the names & all the ips in a rule
def parse_rule(rule):
    hostnames = []
    ips = []
    words = rule.split(" ")
    for word in words:
        word = word.strip("(")
        word = word.strip(")")
        if (is_ip(word)):
            ips.append(word)
        if (is_hostname(word)):
            hostnames.append(word) 
    return (ips, hostnames)

for i in rules_rows:
    rule = rules_pd.loc[i, "Rules"]
    print(rule)
    (ips, hostnames) = parse_rule(rule)
    
    for ip in ips:
        (nslookup_comment, ping_comment) = comment_on_dns(ip)
        rules_nslookup_comments[i] += nslookup_comment
        rules_ping_comments[i] += ping_comment
    for hostname in hostnames:
        (nslookup_comment, ping_comment) = comment_on_dns(hostname)
        rules_nslookup_comments[i] += nslookup_comment
        rules_ping_comments[i] += ping_comment

        
print_dict(dns_records, "DNS")
print_dict(dns_pings, "pings")

print_arr(rules_nslookup_comments, "nslookup comments")
print_arr(rules_ping_comments, "ping comments")
            
rules_pd.insert(1, 'nslookup comments', rules_nslookup_comments)
rules_pd.insert(2, 'ping comments', rules_ping_comments)

rules_pd.to_csv('./new_csv/new_rules.csv')

access-list 51 remark PIM ACL for OFF CAMPUS TRAFFIC
access-list 51 remark * This Line is for Open Student Television Network *
access-list 51 standard permit 233.127.154.0 255.255.255.0
access-list 56 remark PIM ACL for ON CAMPUS TRAFFIC
access-list 56 remark This allows GHOST to Run On Campus Only
access-list 56 standard permit 224.77.0.0 255.255.0.0
access-list internet-in extended permit ip any4 any4 log
access-list internet_access_in extended deny tcp any4 any4 eq sunrpc
access-list internet_access_in extended deny udp any4 any4 eq sunrpc
access-list internet_access_in remark block access to RDP on VPN due to exploits
access-list internet_access_in remark port blocks for BC--added 01/30/14-johnja
access-list internet_access_in extended deny udp any4 any4 object-group blocked-udp-ports
access-list internet_access_in remark Athletics ticket 1776 and 2412
access-list internet_access_in extended permit ip host 146.115.17.37 host 136.167.44.69
access-list internet_access_in extended pe

access-list internet_access_in extended permit ip host 65.61.186.137 object suffolk-asa
access-list internet_access_in remark 6-2-2022 request for 136.167.45.245 for suffolk construction vpn
access-list internet_access_in extended permit ip host 13.92.245.252 host 136.167.45.245
access-list internet_access_in extended permit ip host 209.117.82.196 host 136.167.45.245
access-list internet_access_in extended permit ip host 12.198.176.2 host 136.167.45.245
access-list internet_access_in extended permit ip host 13.90.159.34 host 136.167.45.245
access-list internet_access_in extended permit ip host 13.90.157.106 host 136.167.45.245
access-list internet_access_in extended permit ip 38.88.216.152 255.255.255.248 host 136.167.45.245
access-list internet_access_in extended permit ip host 52.36.70.147 host 136.167.45.245
access-list internet_access_in extended permit ip host 52.25.109.29 host 136.167.45.245
access-list internet_access_in extended permit ip 134.6.202.224 255.255.255.240 host 136.

access-list internet_access_in extended permit tcp host 40.76.70.127 host 136.167.224.28 eq 9001
access-list internet_access_in extended permit tcp host 40.76.70.127 host 136.167.224.28 eq 8000
access-list internet_access_in extended permit tcp host 40.76.70.127 host 136.167.232.51 eq 8000
access-list internet_access_in remark rule to allow 8000 on tz-pad-2063.bc.edu FWRT477
access-list internet_access_in remark Request 1006 - add new source and destination hosts for smart lockers
access-list internet_access_in extended permit tcp host 168.128.152.240 host 136.167.225.171 eq 8000
access-list internet_access_in extended permit tcp host 168.128.152.240 host 136.167.232.176 eq 8000
access-list internet_access_in remark addresses need access to device geometer.bc.edu to permit upload of seismic data IM197615
access-list internet_access_in extended permit udp host 136.177.24.132 host 136.167.83.252 eq 2543
access-list internet_access_in extended permit tcp host 136.177.24.132 host 136.167.8

access-list internet_access_in extended permit tcp host 128.95.166.5 object nesn-edge4.bc.edu eq 16020
access-list internet_access_in remark 8022 access for getz.bc.edu
access-list internet_access_in extended permit tcp any4 object getz.bc.edu eq 2022
access-list internet_access_in remark will gartners video encoder RM956983
access-list internet_access_in extended permit tcp object-group will-video-encoder-hosts host 136.167.45.252 eq 3999
access-list internet_access_in extended permit tcp any4 host 136.167.181.240 eq www
access-list internet_access_in remark other subnets are also blocked
access-list internet_access_in extended deny ip any4 object-group default-deny-subnets
access-list internet_access_in remark block traffic to two St. C's addresses for testing
access-list internet_access_in extended deny ip any4 136.167.27.242 255.255.255.254
access-list internet_access_in remark Block all traffic to machine per SC ticket IM253751 4-Jan-2008
access-list internet_access_in extended de









68.171.146.19 doesn't ping
static-19-146-171-68.axsne.net doesn't ping
136.167.48.3 doesn't ping

68.171.146.19 doesn't ping
static-19-146-171-68.axsne.net doesn't ping
136.167.48.3 doesn't ping





136.167.201.133 doesn't ping
cjis-out.bc.edu doesn't ping

136.167.37.0 doesn't ping
255.255.255.0 doesn't ping


136.167.8.0 doesn't ping
255.255.255.0 doesn't ping








dauntlessnew.bc.edu doesn't ping

dauntlessnew.bc.edu doesn't ping


136.167.117.249 doesn't ping
temp-cs-legacy.bc.edu doesn't ping

136.167.117.249 doesn't ping
temp-cs-legacy.bc.edu doesn't ping

136.167.117.249 doesn't ping
temp-cs-legacy.bc.edu doesn't ping

136.167.117.249 doesn't ping
temp-cs-legacy.bc.edu doesn't ping





136.167.173.190 doesn't ping


128.97.94.157 doesn't ping
gbormd.igpp.ucla.edu doesn't ping

128.97.94.157 doesn't ping
gbormd.igpp.ucla.edu doesn't ping

128.97.94.159 doesn't ping
gmag-reserved.igpp.ucla.edu doesn't ping

128.97.94.159 doesn't ping
gmag-reserved.igpp.ucla.edu doesn'

In [25]:
#testing random stuff below

def parse_file(filename):
    file = open(filename)
    lines = file.readlines()

    
for line in lines:
    print(line)
    
filename = './print_results/rules_printing.txt'

access-list 51 remark PIM ACL for OFF CAMPUS TRAFFIC

access-list 51 remark * This Line is for Open Student Television Network *

access-list 51 standard permit 233.127.154.0 255.255.255.0

access-list 56 remark PIM ACL for ON CAMPUS TRAFFIC

access-list 56 remark This allows GHOST to Run On Campus Only

access-list 56 standard permit 224.77.0.0 255.255.0.0

access-list internet-in extended permit ip any4 any4 log

access-list internet_access_in extended deny tcp any4 any4 eq sunrpc

access-list internet_access_in extended deny udp any4 any4 eq sunrpc

access-list internet_access_in remark block access to RDP on VPN due to exploits

access-list internet_access_in remark port blocks for BC--added 01/30/14-johnja

access-list internet_access_in extended deny udp any4 any4 object-group blocked-udp-ports

access-list internet_access_in remark Athletics ticket 1776 and 2412

access-list internet_access_in extended permit ip host 146.115.17.37 host 136.167.44.69

access-list internet_access_


136.167.8.110              False

64.57.22.202               True

64.57.22.218               True

64.57.22.217               True

64.57.22.194               True

64.57.22.195               True

64.57.22.210               True

64.57.22.211               True

64.57.22.197               True

mail.sip.net.internet2.edu True

64.57.22.213               True

64.57.22.198               True

64.57.22.214               True

64.57.22.199               True

64.57.22.215               True

72.48.119.220              False

72-48-119-220.static.grandenetworks.netFalse

136.167.36.0               False

162.211.108.162            True

162.211.108.162.hvdata.net False

50.206.156.130             True

136.167.235.254            False

eopss-2.bc.edu             False

136.167.231.236            False

136.167.231.235            False

malojd.bc.edu              False

136.167.201.133            False

cjis-out.bc.edu            False

170.223.168.132            False

ewanr1-g-0-1.nete

In [32]:
def split_string(word):
    num_letters = len(word)
    for i in range

def get_dns(line):
    words = line.split()
    if len(words) == 2:
        return (words[0], words[1])
    else:
        new_words = line.split(".")
        num_words = len(new_words)
        octets = [new_words[i] for i in range(num_words-4,num_words)]
        
def get_pings(line):
    if line[-4:] == "True":
        return True
    return False

nw = [1,2,3,4,5,6,7]
octets = [nw[i] for i in range(len(nw)-4,len(nw))]
print(octets)

 True
[4, 5, 6, 7]
